In [ ]:
import json
import dotenv
import os
from azure.identity import DefaultAzureCredential
from openai import AzureOpenAI
import requests
from IPython.display import Image, display

dotenv.load_dotenv()

endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
deployment = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT")
api_key = os.environ.get("AZURE_OPENAI_KEY")
api_version = os.environ.get("AZURE_OPENAI_API_VERSION")
deployment_embedding = os.environ.get("AZURE_OPENAI_CHATGPT_EMBEDDING_DEPLOYMENT")

search_endpoint = os.environ.get("SEARCH_ENDPOINT")
search_index = os.environ.get("SEARCH_INDEX")
search_api_key = os.environ.get("SEARCH_API_KEY")
search_semantic_config = os.environ.get("SEARCH_SEMANTIC_CONFIGURATION")

In [ ]:

creds = DefaultAzureCredential()
token = creds.get_token("https://storage.azure.com/.default").token

In [ ]:
aoai_client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version=api_version,
    )

In [ ]:
search_text = 'Tell me about Kubernetes.'

In [ ]:
system_prompt = "You are an AI assistant that helps people find information."

completion = aoai_client.chat.completions.create(
    model=deployment,
    messages=[
        {
            "role": "user",
            "content": f"{search_text}"
        }
    ],
    extra_body={
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "semantic_configuration": search_semantic_config,
                    "role_information": system_prompt,
                    "endpoint": search_endpoint,
                    "index_name": search_index,
                    "key": search_api_key,
                    "embedding_dependency": {
                        "type": "deployment_name",
                        "deployment_name": deployment_embedding
                    }
                },
            }
        ]
    },
)

result = json.loads(completion.to_json())

In [ ]:
images_list = []

print('*** LLM generated response content: ***\n')
print(result['choices'][0]['message']['content'])

print('\n\n*** Citations: ***\n')
if result['choices'][0]['message']['context']['citations']:
    for citation in result['choices'][0]['message']['context']['citations']:
        print(f"Content: {citation['content']}")
        print(f"File URL: {citation['filepath']}")
        images = json.loads(citation['url'])
        for image in images:
            print(f"Image URL: {image}")
            images_list.append(image)
        print()

print(f'\n\n*** Citations have in total {len(images_list)} images. ***\n')
for img in images_list:
    print(img)
    r = requests.get(img, headers={ "Authorization": "Bearer " + token, 'x-ms-version': '2020-04-08'})
    display(Image(r.content, width=640))